In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship

In [2]:
# Change Column Names of API output to not include parentheses
file = "chinese_drop_duplicates_in_Austin_TX.csv"
restaurants_df = pd.read_csv(file)
restaurants_df = restaurants_df.rename(columns={"ID(Google)":"Google_ID", "Price_Level(Google)":"Price_Level", "Rating(Google)":"Rating"})
restaurants_df.head()

,Address,Google_ID,Latitude,Longitude,Name,Price_Level,Rating,Zip
0,"1931 E Ben White Blvd #300, Austin, TX 78741, USA",b9eb17dab1edbc2b1a046a224789b1330e44554c,30.215372,-97.744803,Asia,2.0,3.8,78741
1,"2712 Bee Caves Rd, Austin, TX 78746, USA",bdefc39e9e72060c0c3f8552220f8ea2ca60d20b,30.271814,-97.786538,Chinatown,2.0,4.1,78746
2,"212 E Oltorf St, Austin, TX 78704, USA",b75b51402eec73943491ad3847b6b1c46d7a0ad8,30.238337,-97.751720,New Mandarin,2.0,3.5,78704
3,"3601, 6605 Airport Blvd, Austin, TX 78752, Uni...",aeca550acd577ada947d15388018945bbcd9b492,30.330697,-97.715309,China Palace,2.0,3.9,78752
4,"625 W Ben White Blvd, Austin, TX 78704, USA",ec2339d24cecd5986b306474bb427557f034a055,30.225787,-97.771176,Bamboo Garden Chinese Restaurant,2.0,4.0,78704


In [3]:
# Getting the city, state and cuisine from the file name so we can put it into a new column
word_list = file.split("_")
cuisine = word_list[0]
city = word_list[4].lower()
end = word_list[5]
second_word_list = end.split(".")
state = second_word_list[0].lower()
new_column = city + "-" + state + "-" + cuisine
new_column

'austin-tx-chinese'

In [4]:
restaurants_df["City_State_Cuisine"] = new_column
restaurants_df

,Address,Google_ID,Latitude,Longitude,Name,Price_Level,Rating,Zip,City_State_Cuisine
0,"1931 E Ben White Blvd #300, Austin, TX 78741, USA",b9eb17dab1edbc2b1a046a224789b1330e44554c,30.215372,-97.744803,Asia,2.0,3.8,78741,austin-tx-chinese
1,"2712 Bee Caves Rd, Austin, TX 78746, USA",bdefc39e9e72060c0c3f8552220f8ea2ca60d20b,30.271814,-97.786538,Chinatown,2.0,4.1,78746,austin-tx-chinese
2,"212 E Oltorf St, Austin, TX 78704, USA",b75b51402eec73943491ad3847b6b1c46d7a0ad8,30.238337,-97.751720,New Mandarin,2.0,3.5,78704,austin-tx-chinese
3,"3601, 6605 Airport Blvd, Austin, TX 78752, Uni...",aeca550acd577ada947d15388018945bbcd9b492,30.330697,-97.715309,China Palace,2.0,3.9,78752,austin-tx-chinese
4,"625 W Ben White Blvd, Austin, TX 78704, USA",ec2339d24cecd5986b306474bb427557f034a055,30.225787,-97.771176,Bamboo Garden Chinese Restaurant,2.0,4.0,78704,austin-tx-chinese
5,"10901 N Lamar Blvd, Austin, TX 78753, USA",f7ba6a5b04d098b3265bd2dbb954b1495292b8dc,30.377804,-97.687811,New Fortune Chinese Seafood Restaurant,2.0,4.3,78753,austin-tx-chinese
6,"4815 W Braker Ln, Austin, TX 78759, USA",b065f06f65bee85cb70cf65ddad37e86d41a8ab6,30.400153,-97.748023,Twin Lion,2.0,4.3,78759,austin-tx-chinese
7,"603 W Stassney Ln, Austin, TX 78745, USA",ae9093c0f3fc4e3e1b64476a888b716a68390ad3,30.207895,-97.780976,1st Wok,2.0,4.4,78745,austin-tx-chinese
8,"107 W 5th St, Austin, TX 78701, USA",e259af5fd78a9780e9ca0ac710946b8ad1bd0638,30.267228,-97.744187,Chinatown,2.0,3.9,78701,austin-tx-chinese
9,"3407 Greystone Dr, Austin, TX 78731, USA",9d64be01b91f73c8eca5c26bea3a3bff79404ffb,30.356586,-97.747509,Chinatown,2.0,4.0,78731,austin-tx-chinese


In [5]:
# Output Cleaned CSV
restaurants_df.to_csv("cleaned"+file)

In [6]:
engine = create_engine("sqlite:///yummydata.sqlite")

In [7]:
Base = declarative_base()

In [8]:
# Create ORM Class for Zip Code Demographic Data
class ZipDemo(Base):
    
    __tablename__ = 'zip_demographics'

    Zip = Column(Integer, primary_key=True)
    Population = Column(Integer)
    Density = Column(Float)
    AverageIncome = Column(Float)
    City = Column(Text)
    State = Column(Text)
    ZipLatitude = Column(Float)
    ZipLongitude = Column(Float)
    City_State = Column(Text, ForeignKey("user_input.City_State"))
    
#     def __repr__(self):
#         return f"id={self.id}, name={self.station}"

In [9]:
class RestaurantSearch(Base):
    
    __tablename__ = 'restaurant_search'

    Address = Column(Text)
    Google_ID = Column(Text, primary_key=True)
    Latitude = Column(Float)
    Longitude = Column(Float)
    Name = Column(Text)
    Price_Level = Column(Integer)
    Rating = Column(Float)
    Zip = Column(Integer, ForeignKey("zip_demographics.Zip"))
    City_State_Cuisine = Column(Text)

In [10]:
class Input(Base):
    
    __tablename__ = 'user_input'

    id = Column(Integer, primary_key=True)
    City = Column(Text)
    State = Column(Text)
    Cuisine = Column(Text)
    City_State = Column(Text)
    City_State_Cuisine = Column(Text)

In [11]:
Base.metadata.create_all(engine)

In [12]:
# Create a Functin to Populate Tables
def populate_table(engine, table, csvfile):
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)

In [13]:
# Populate Tables
populate_table(engine, ZipDemo.__table__, 'zip_demographicchineseATX.csv')
populate_table(engine, RestaurantSearch.__table__, 'cleaned'+file)